In [1]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# !pip install --upgrade segmentation_models_pytorch

In [2]:
import os
import shutil
import pathlib
import gc
import pandas as pd
import ttach as tta
from pathlib import Path
from PIL import Image
import numpy as np
import cv2 as cv
import random
import matplotlib.pyplot as plt
# from tqdm.auto import tqdm
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torch
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset

import torchvision
from torchvision import datasets
import cv2
from torch.cuda import amp

import torchvision.transforms as T
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.utils import make_grid
import optuna

import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
import timm

In [3]:
def seed_everything(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

In [4]:
seed_everything(42)

In [5]:

ensemple_type = "No" #"No" #"mine"

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [7]:
import torchvision.transforms as T


class ContrailDataset:
    def __init__(self, df, transform=None, normalize=False):
        self.df = df  
        self.images = df['image']
        self.labels = df['label']
        self.transform =transform
        self.normalize_image = T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        self.normalize=normalize
        
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        image = np.load("../../input/" + self.images[idx]).astype(float)   
        label = np.load("../../input/" + self.labels[idx]).astype(float)
        
        
        # label_cls = 1 if label.sum() > 0 else 0
        if self.transform :
            data = self.transform(image=image, mask=label)
            image  = data['image']
            label  = data['mask']
            image = np.transpose(image, (2, 0, 1))
            label = np.transpose(label, (2, 0, 1))    
            
            
#         return torch.tensor(image), torch.tensor(label)
    
        if self.normalize:
            image = self.normalize_image(torch.tensor(image))
            return image, torch.tensor(label)
        else:
            return torch.tensor(image), torch.tensor(label)
    
    

In [8]:
###TImmunetplusplus model Nirjhar

n_blocks = 4

class TimmSegModel(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(6, 12, 3, stride=1, padding=1, bias=False)
        self.conv3 = nn.Conv2d(12, 36, 3, stride=1, padding=1, bias=False)
        self.mybn1 = nn.BatchNorm2d(6)
        self.mybn2 = nn.BatchNorm2d(12)
        self.mybn3 = nn.BatchNorm2d(36)     
        self.encoder = timm.create_model(
            cfg['backbone'],
            in_chans=3,
            features_only=True,
            drop_rate=0.8,
            drop_path_rate=0.5,
            pretrained=False
        )
        self.encoder.conv_stem=nn.Conv2d(6, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        self.encoder.blocks[5] = nn.Identity()
        self.encoder.blocks[6] = nn.Sequential(
            nn.Conv2d(self.encoder.blocks[4][2].conv_pwl.out_channels, 320, 1),
            nn.BatchNorm2d(320),
            nn.ReLU6(),
        )
        tr = torch.randn(1,6,64,64)
        g = self.encoder(tr)
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Conv2d(
            decoder_channels[n_blocks-1],
            cfg["num_classes"], 
            kernel_size=(3, 3),
            stride=(1, 1), 
            padding=(1, 1)
        )

    def forward(self,x):
        x = F.relu6(self.mybn1(self.conv1(x)))
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    

def load_model1(cfg):
    model = TimmSegModel(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model


#### Model 2 Nirjhar

n_blocks =4
class TimmSegModel2(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel2, self).__init__()

        self.encoder = timm.create_model(
            cfg['backbone'],
            in_chans=3,
            features_only=True,
            drop_rate=0.5,
            pretrained=False
        )
        g = self.encoder(torch.rand(1, 3, 128, 128))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Sequential(
            nn.Conv2d(decoder_channels[n_blocks-1], 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.UpsamplingBilinear2d(scale_factor=1))

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    

def load_model2(cfg):
    model = TimmSegModel2(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model


##################

n_blocks = 4

class TimmSegModel3(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModel3, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(6, 12, 3, stride=1, padding=1, bias=False)
        self.conv3 = nn.Conv2d(12, 36, 3, stride=1, padding=1, bias=False)
        self.mybn1 = nn.BatchNorm2d(6)
        self.mybn2 = nn.BatchNorm2d(12)
        self.mybn3 = nn.BatchNorm2d(36)     
        self.encoder = timm.create_model(
            cfg["backbone"],
            in_chans=3,
            features_only=True,
            drop_rate=0.8,
            drop_path_rate=0.5,
            pretrained=False
        )
        self.encoder.conv_stem=nn.Conv2d(6, 72, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

        self.encoder.blocks[5] = nn.Identity()
        self.encoder.blocks[6] = nn.Sequential(
            nn.Conv2d(self.encoder.blocks[4][2].conv_pwl.out_channels, 320, 1),
            nn.BatchNorm2d(320),
            nn.ReLU6(),
        )
        tr = torch.randn(1,6,64,64)
        g = self.encoder(tr)
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Conv2d(decoder_channels[n_blocks-1], cfg['num_classes'], kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

    def forward(self,x):
        x = F.relu6(self.mybn1(self.conv1(x)))
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features
    
    
    
def load_model3(cfg):
    model = TimmSegModel3(cfg)
    model.load_state_dict(torch.load(cfg['model_pth']))
    return model



## Rohit Model 
class Net(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        
        self.model = smp.Unet(
            encoder_name=cfg["backbone"],     
            encoder_weights=None,   
            in_channels=3,  
            classes=cfg["num_classes"],
            activation=None
        )
        
    
    def forward(self, inputs):
        mask = self.model(inputs)
        return mask
    
    
def load_modelr1(cfg):
    model = Net(cfg)
    model = torch.nn.DataParallel(model).cuda()
    model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
    return model


#     if idx in [11, 12]:
#         model = TimmSegModelR1(cfg)
#         model = torch.nn.DataParallel(model).cuda()
#         model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
#     else:
#         model = Net(cfg)    
#         model = torch.nn.DataParallel(model).cuda()
#         model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])

class TimmSegModelR1(nn.Module):
    def __init__(self, cfg, segtype='unet', pretrained=True):
        super(TimmSegModelR1, self).__init__()

        self.n_blocks = 4
        self.encoder = timm.create_model(
            cfg['backbone'],
            in_chans=3,
            features_only=True,
            drop_rate=0.5,
            pretrained=False
        )
        g = self.encoder(torch.rand(1, 3, 128, 128))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.decoders.unetplusplus.decoder.UnetPlusPlusDecoder(
                encoder_channels=encoder_channels[:self.n_blocks+1],
                decoder_channels=decoder_channels[:self.n_blocks],
                n_blocks=self.n_blocks,
            )

        self.segmentation_head = nn.Sequential(
            nn.Conv2d(decoder_channels[self.n_blocks-1], 
                      cfg['num_classes'],
                      kernel_size=(3, 3),
                      stride=(1, 1),
                      padding=(1, 1)
            ),
            nn.UpsamplingBilinear2d(scale_factor=1)
        )

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:self.n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features


    
def load_modelr2(cfg):
    model = TimmSegModelR1(cfg)
    model = torch.nn.DataParallel(model).cuda()
    model.load_state_dict(torch.load(cfg['model_pth'], map_location=torch.device('cpu'))['model'])
    return model

In [9]:
def dice_coef(y_true, y_pred, thr=0.5, epsilon=1e-6):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum()
    den = y_true.sum() + y_pred.sum()
    dice = ((2*inter+epsilon)/(den+epsilon)).mean()
    
    return dice


def get_transform(img_size):
    transform = A.Compose([
        A.Resize(*img_size, interpolation=cv2.INTER_NEAREST),
    ], p=1.0)
    return transform

In [10]:
val_df = pd.read_csv("../../input/data_utils/val_df_filled.csv")
val_dups = np.load("../../input/data_utils/dups_val.npy")

val_dups = [int(val_id) for val_id in val_dups]

In [11]:
val_df = val_df.loc[~val_df['id'].isin(val_dups)].reset_index(drop=True)
print(val_df.shape)

(1633, 4)


In [12]:
# 1 nir_01
# 2 nir_02
# 3 nir_04
# 4 nir_05_tta
# 5 nir_06
# 6 nir_08

#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b7_ns',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/qishentrialv512unetplus_tf_efficientnet_b7_ns_best_epochstage2cv669-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_01", 
#         'model_func': load_model1,
#         'tta': False
#     },
#     {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l1',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/unetplusecarnf01_eca_nfnet_l1_best_epochstage2cv656-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_02",
#         'model_func': load_model2,
#         'tta': False
#     },

#     {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l1',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl1v2fold0_eca_nfnet_l1_best_epoch-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_04",
#         'model_func': load_model2,
#         'tta': False
#     }, # w8
#     {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l2',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl2v2fold0_eca_nfnet_l2_best_epochstage2-00.bin',
#         'threshold': 0.62, #0.24,
#         'call_sign': "nir_05_tta",
#         'model_func': load_model2,
#         'tta': True
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'tf_efficientnet_b8',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_06", 
#         'model_func': load_model3,
#         'tta': False
#     },
#     {
#         'model_name': 'Unet++',
#         'backbone': 'densenetblur121d',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl2v2fold0_densenetblur121d_best_epochstage2-00.bin',
#         'threshold': 0.79,
#         'call_sign': "nir_08", 
#         'model_func': load_model2,
#         'tta': False
#     }





In [13]:
CFGS1 = [ 
    {
        'model_name': 'Unet++',
        'backbone': 'tf_efficientnet_b7_ns',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/qishentrialv2fpn512_tf_efficientnet_b7_ns_best_epochcv650lb675-00.bin',
        'threshold': 0.62, #0.24,
        'call_sign': "nir_tta_sig",
        'model_func': load_model1,
        'tta': True
    },
    
    {
        'model_name': 'Unet++',
        'backbone': 'tf_efficientnet_b7_ns',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/qishentrialv2fpn512_tf_efficientnet_b7_ns_best_epochcv650lb675-00.bin',
        'threshold': 0.62, #0.24,
        'call_sign': "nir_sig",
        'model_func': load_model1,
        'tta': False
    },
    
    
    {
        'model_name': 'Unet++',
        'backbone': 'tf_efficientnet_b7_ns',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/qishentrialv512unetplus_tf_efficientnet_b7_ns_best_epochstage2cv669-00.bin',
        'threshold': 0.62, #0.24,
        'call_sign': "nir_01_tta_sig", 
        'model_func': load_model1,
        'tta': True
    },
    
    {
        'model_name': 'Unet++',
        'backbone': 'tf_efficientnet_b7_ns',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/qishentrialv512unetplus_tf_efficientnet_b7_ns_best_epochstage2cv669-00.bin',
        'threshold': 0.62, #0.24,
        'call_sign': "nir_01_sig", 
        'model_func': load_model1,
        'tta': False
    },
    
    {
        'model_name': 'Unet',
        'backbone': 'eca_nfnet_l1',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/unetplusecarnf01_eca_nfnet_l1_best_epochstage2cv656-00.bin',
        'threshold': 0.62, #0.24,
        'call_sign': "nir_02_tta_sig",
        'model_func': load_model2,
        'tta': True
    },
    
    {
        'model_name': 'Unet',
        'backbone': 'eca_nfnet_l1',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/contrail_01/output/nirjhar/unetplusecarnf01_eca_nfnet_l1_best_epochstage2cv656-00.bin',
        'threshold': 0.62, #0.24,
        'call_sign': "nir_02_sig",
        'model_func': load_model2,
        'tta': False
    },
    
    {
        'model_name': 'Unet',
        'backbone': 'eca_nfnet_l1',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl1v2fold0_eca_nfnet_l1_best_epochstage2-00.bin',
        'threshold': 0.62, #0.24,
        'call_sign': "nir_03_tta_sig",
        'model_func': load_model2,
        'tta': True
    },
    
    {
        'model_name': 'Unet',
        'backbone': 'eca_nfnet_l1',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl1v2fold0_eca_nfnet_l1_best_epochstage2-00.bin',
        'threshold': 0.62, #0.24,
        'call_sign': "nir_03_sig",
        'model_func': load_model2,
        'tta': False
    },
    
    {
        'model_name': 'Unet',
        'backbone': 'eca_nfnet_l1',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl1v2fold0_eca_nfnet_l1_best_epoch-00.bin',
        'threshold': 0.62, #0.24,
        'call_sign': "nir_04_tta_sig",
        'model_func': load_model2,
        'tta': True
    },
    
        {
        'model_name': 'Unet',
        'backbone': 'eca_nfnet_l1',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl1v2fold0_eca_nfnet_l1_best_epoch-00.bin',
        'threshold': 0.62, #0.24,
        'call_sign': "nir_04_sig",
        'model_func': load_model2,
        'tta': False
    }, # w8
    
    {
        'model_name': 'Unet',
        'backbone': 'eca_nfnet_l2',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl2v2fold0_eca_nfnet_l2_best_epochstage2-00.bin',
        'threshold': 0.62, #0.24,
        'call_sign': "nir_05_tta_sig",
        'model_func': load_model2,
        'tta': True
    },
        {
        'model_name': 'Unet',
        'backbone': 'eca_nfnet_l2',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl2v2fold0_eca_nfnet_l2_best_epochstage2-00.bin',
        'threshold': 0.62, #0.24,
        'call_sign': "nir_05_sig",
        'model_func': load_model2,
        'tta': False
    },
    
    
    {
    'model_name': 'Unet++',
        'backbone': 'tf_efficientnet_b8',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2-00.bin',
        'threshold': 0.79,
        'call_sign': "nir_06_tta_sig", 
        'model_func': load_model3,
        'tta': True
    },  # 11
    
        {
    'model_name': 'Unet++',
        'backbone': 'tf_efficientnet_b8',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/effb8modelv2fold0_tf_efficientnet_b8_best_epochstage2-00.bin',
        'threshold': 0.79,
        'call_sign': "nir_06_sig", 
        'model_func': load_model3,
        'tta': False
    }, 
    
    {
    'model_name': 'Unet++',
        'backbone': 'tf_efficientnet_b8',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/effb8modelv2fold0_tf_efficientnet_b8_best_epoch-00.bin',
        'threshold': 0.79,
        'call_sign': "nir_07_tta_sig", 
        'model_func': load_model3,
        'tta': True
    },
    
        {
    'model_name': 'Unet++',
        'backbone': 'tf_efficientnet_b8',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/effb8modelv2fold0_tf_efficientnet_b8_best_epoch-00.bin',
        'threshold': 0.79,
        'call_sign': "nir_07_sig", 
        'model_func': load_model3,
        'tta': False
    },
    
    {
    'model_name': 'Unet++',
        'backbone': 'densenetblur121d',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl2v2fold0_densenetblur121d_best_epochstage2-00.bin',
        'threshold': 0.79,
        'call_sign': "nir_08_tta_sig", 
        'model_func': load_model2,
        'tta': True
    },
    {
    'model_name': 'Unet++',
        'backbone': 'densenetblur121d',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl2v2fold0_densenetblur121d_best_epochstage2-00.bin',
        'threshold': 0.79,
        'call_sign': "nir_08_sig", 
        'model_func': load_model2,
        'tta': False
    },
    
    
   {
    'model_name': 'Unet++',
        'backbone': 'convnextv2_base',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/convnextfold0_convnextv2_base_best_epochstage2-00.bin',
        'threshold': 0.79,
        'call_sign': "nir_09_tta_sig", 
        'model_func': load_model2,
        'tta': True
    },
    {
    'model_name': 'Unet++',
        'backbone': 'convnextv2_base',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/nirjhar/convnextfold0_convnextv2_base_best_epochstage2-00.bin',
        'threshold': 0.79,
        'call_sign': "nir_09_sig", 
        'model_func': load_model2,
        'tta': False
    }
    
    
    
    
]

In [14]:
final_preds1 = []

for idx, cfg in enumerate(CFGS1):   

    if idx <= 7:
        continue
        
    print(cfg)
    val_transform = get_transform(cfg['img_size'])
    valid_dataset = ContrailDataset(val_df, transform=val_transform, normalize=False)  
    
    valid_loader = DataLoader(
        valid_dataset, 
        batch_size = 32, #32, 
        shuffle = False, 
        num_workers = 2, 
        pin_memory = True, 
        drop_last = False
    )
    
    model = cfg['model_func'](cfg)        
      
    if cfg['tta']:
        model = tta.SegmentationTTAWrapper(model, tta.aliases.hflip_transform(), merge_mode='mean')
    
    
    model.to(device)
    model.eval()
    
    preds = []
    masks_ = []      
            

    for index, (images, masks) in enumerate(tqdm(valid_loader)):  
        images  = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        if cfg['img_size'][0] != 256:
            masks = torch.nn.functional.interpolate(masks, size=256, mode='nearest') 

        masks_.append(torch.squeeze(masks, dim=1))
        with torch.inference_mode():
            images = torch.nn.functional.interpolate(images, size=cfg['img_size'][0], mode='nearest')
            pred = model(images).sigmoid()   
            pred = torch.nn.functional.interpolate(pred, size=256, mode='nearest') 
            preds.append(torch.squeeze(pred, dim=1))
        
            
            
    model_masks = torch.cat(masks_, dim=0)
    model_preds = torch.cat(preds, dim=0)
        
    model_masks = torch.flatten(model_masks, start_dim=0, end_dim=1)
    model_preds = torch.flatten(model_preds, start_dim=0, end_dim=1)  
    
    # save
    torch.save(model_preds, f"../../output/final_preds/{cfg['call_sign']}.pt")    
#     if idx == 6:
#     torch.save(model_masks, f'../../output/final_preds/val_masks0.pt')

    
    best_threshold = 0.0
    best_dice_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
#         score = dice_coef(model_masks, model_preds.sigmoid(), thr=threshold).cpu().detach().numpy() 
        score = dice_coef(model_masks, model_preds, thr=threshold).cpu().detach().numpy() 

        
        if score > best_dice_score:
            best_dice_score = score
            best_threshold = threshold
    
        
    print(best_dice_score, best_threshold)
    final_preds1.append(model_preds)
    
    del model
    torch.cuda.empty_cache()
    gc.collect()

{'model_name': 'Unet', 'backbone': 'eca_nfnet_l1', 'img_size': [512, 512], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl1v2fold0_eca_nfnet_l1_best_epoch-00.bin', 'threshold': 0.62, 'call_sign': 'nir_04_tta_sig', 'model_func': <function load_model2 at 0x7fc005599a20>, 'tta': True}


  0%|          | 0/52 [00:00<?, ?it/s]

0.6502031 0.35
{'model_name': 'Unet', 'backbone': 'eca_nfnet_l1', 'img_size': [512, 512], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl1v2fold0_eca_nfnet_l1_best_epoch-00.bin', 'threshold': 0.62, 'call_sign': 'nir_04_sig', 'model_func': <function load_model2 at 0x7fc005599a20>, 'tta': False}


  0%|          | 0/52 [00:00<?, ?it/s]

0.6516042 0.52
{'model_name': 'Unet', 'backbone': 'eca_nfnet_l2', 'img_size': [512, 512], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/Contrail_Detection/output/nirjhar/ecanfnetl2v2fold0_eca_nfnet_l2_best_epochstage2-00.bin', 'threshold': 0.62, 'call_sign': 'nir_05_tta_sig', 'model_func': <function load_model2 at 0x7fc005599a20>, 'tta': True}


...

0.64298403 0.01
{'model_name': 'Unet++', 'backbone': 'convnextv2_base', 'img_size': [512, 512], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/Contrail_Detection/output/nirjhar/convnextfold0_convnextv2_base_best_epochstage2-00.bin', 'threshold': 0.79, 'call_sign': 'nir_09_tta_sig', 'model_func': <function load_model2 at 0x7fc005599a20>, 'tta': True}


  0%|          | 0/52 [00:00<?, ?it/s]

0.65553015 0.01
{'model_name': 'Unet++', 'backbone': 'convnextv2_base', 'img_size': [512, 512], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/Contrail_Detection/output/nirjhar/convnextfold0_convnextv2_base_best_epochstage2-00.bin', 'threshold': 0.79, 'call_sign': 'nir_09_sig', 'model_func': <function load_model2 at 0x7fc005599a20>, 'tta': False}


  0%|          | 0/52 [00:00<?, ?it/s]

0.64687884 0.01


In [14]:


#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold3.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01_tta",
#         'model_func': load_modelr1,
#         'tta': True
#     }, 

#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold3.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_01",
#         'model_func': load_modelr1,
#         'tta': False
#     }, 
    
#         {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [384, 384],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb/Unet/efficientnet-b7-384/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'model_func': load_modelr1,
#         'call_sign': "roh_03_tta",
#         'tta': True
#     },
        
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_02_pl_687lb_fulltrain/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_04_tta",
#         'model_func': load_modelr1,
#         'normalize': True,
#         'tta': True
#     },
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_02_pl_687lb_fulltrain/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_04",
#         'model_func': load_modelr1,
#         'normalize': True,
#         'tta': False
#     },
    
#         {
#         'model_name': 'Unet',
#         'backbone': 'eca_nfnet_l0',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_03_pl_687lb_fulltrain/Unet/eca_nfnet_l0-256/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_05",
#         'model_func': load_modelr2,
#         'normalize': True,
#         'tta': False
#     },
        
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [512, 512],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_06_tta",
#         'model_func': load_modelr1,
#         'normalize': False,
#         'tta': True
#     },



In [19]:
CFGS2 = [     
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold3.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_01_tta_sig",
        'model_func': load_modelr1,
        'tta': True
    }, 
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_s2/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold3.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_01_sig",
        'model_func': load_modelr1,
        'tta': False
    }, 
    
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'model_func': load_modelr1,
        'call_sign': "roh_02_tta_sig",
        'tta': True
    },   
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'model_func': load_modelr1,
        'call_sign': "roh_02_sig",
        'tta': False
    },   
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [384, 384],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb/Unet/efficientnet-b7-384/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'model_func': load_modelr1,
        'call_sign': "roh_03_tta_sig",
        'tta': True
    },
    
        {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [384, 384],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb/Unet/efficientnet-b7-384/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'model_func': load_modelr1,
        'call_sign': "roh_03_sig",
        'tta': False
    },
    
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_02_pl_687lb_fulltrain/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_04_tta_sig",
        'model_func': load_modelr1,
        'normalize': True,
        'tta': True
    },
    
        {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_02_pl_687lb_fulltrain/Unet/efficientnet-b7-256/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_04_sig",
        'model_func': load_modelr1,
        'normalize': True,
        'tta': False
    },
    
    ### above this are final    
    {
        'model_name': 'Unet',
        'backbone': 'eca_nfnet_l0',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_03_pl_687lb_fulltrain/Unet/eca_nfnet_l0-256/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_05_tta_sig",
        'model_func': load_modelr2,
        'normalize': True,
        'tta': True
    },
    
    {
        'model_name': 'Unet',
        'backbone': 'eca_nfnet_l0',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_03_pl_687lb_fulltrain/Unet/eca_nfnet_l0-256/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_05_sig",
        'model_func': load_modelr2,
        'normalize': True,
        'tta': False
    },
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_06_tta_sig",
        'model_func': load_modelr1,
        'normalize': False,
        'tta': True
    },
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_06_sig",
        'model_func': load_modelr1,
        'normalize': False,
        'tta': False
    },
    
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb_s2/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_07_tta_sig",
        'model_func': load_modelr1,
        'normalize': False,
        'tta': True
    },
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_687lb_s2/Unet/efficientnet-b7-512/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'call_sign': "roh_07_sig",
        'model_func': load_modelr1,
        'normalize': False,
        'tta': False
    },
    
    ### already done
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [384, 384],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_s2/Unet/efficientnet-b7-384/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'model_func': load_modelr1,
        'call_sign': "roh_08_tta_sig",
        'tta': True
    },
    
    {
        'model_name': 'Unet',
        'backbone': 'efficientnet-b7',
        'img_size': [384, 384],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl_s2/Unet/efficientnet-b7-384/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'model_func': load_modelr1,
        'call_sign': "roh_08_sig",
        'tta': False
    },
    
    
    
    {
        'model_name': 'Unet',
        'backbone': 'tf_mixnet_l',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01/Unet/tf_mixnet_l-512/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'model_func': load_modelr2,
        'call_sign': "roh_09_tta_sig",
        'tta': True
    },
    
    
    {
        'model_name': 'Unet',
        'backbone': 'tf_mixnet_l',
        'img_size': [512, 512],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01/Unet/tf_mixnet_l-512/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'model_func': load_modelr2,
        'call_sign': "roh_09_sig",
        'tta': False
    },
    
    {
        'model_name': 'Unet',
        'backbone': 'fbnetv3_g',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01/Unet/fbnetv3_g-256/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'model_func': load_modelr2,
        'call_sign': "roh_10_tta_sig",
        'tta': True
    },
    
    
    {
        'model_name': 'Unet',
        'backbone': 'fbnetv3_g',
        'img_size': [256, 256],
        'num_classes': 1,
        'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01/Unet/fbnetv3_g-256/checkpoint_dice_ctrl_fold0.pth',
        'threshold': 0.24, #0.24,
        'model_func': load_modelr2,
        'call_sign': "roh_10_sig",
        'tta': False
    },
    
    
    
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl/Unet/efficientnet-b7-256/checkpoint_dice_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_08_tta",
#         'model_func': load_modelr1,
#         'normalize': False,
#         'tta': True
#     },
    
#     {
#         'model_name': 'Unet',
#         'backbone': 'efficientnet-b7',
#         'img_size': [256, 256],
#         'num_classes': 1,
#         'model_pth':  '/home/rohits/pv1/Contrail_Detection/output/exp_01_pl/Unet/efficientnet-b7-256/checkpoint_dice_fold0.pth',
#         'threshold': 0.24, #0.24,
#         'call_sign': "roh_08",
#         'model_func': load_modelr1,
#         'normalize': False,
#         'tta': False
#     },
   
]


In [20]:
final_preds2 = []

for idx, cfg in enumerate(CFGS2):   

    if idx <= 15:
        continue
        
    print(cfg)
    val_transform = get_transform(cfg['img_size'])
    valid_dataset = ContrailDataset(val_df, transform=val_transform, normalize=False)  
    valid_dataset2 = ContrailDataset(val_df, transform=val_transform, normalize=True)  

    
    valid_loader = DataLoader(
        valid_dataset, 
        batch_size = 32, #32, 
        shuffle = False, 
        num_workers = 2, 
        pin_memory = True, 
        drop_last = False
    )
    
    
    valid_loader2 = DataLoader(
        valid_dataset2, 
        batch_size = 32, #32, 
        shuffle = False, 
        num_workers = 2, 
        pin_memory = True, 
        drop_last = False
    )

    model = cfg['model_func'](cfg)  
    if cfg['tta']:
        model = tta.SegmentationTTAWrapper(model, tta.aliases.flip_transform(), merge_mode='mean')

    model.to(device)
    model.eval()
    
    preds = []
    masks_ = []      
            
    if "normalize" in cfg and cfg['normalize']:
        
        for index, (images, masks) in enumerate(tqdm(valid_loader2)):  
            images  = images.to(device, dtype=torch.float)
            masks  = masks.to(device, dtype=torch.float)
            if cfg['img_size'][0] != 256:
                masks = torch.nn.functional.interpolate(masks, size=256, mode='nearest') 

            masks_.append(torch.squeeze(masks, dim=1))

            with torch.inference_mode():
                images = torch.nn.functional.interpolate(images, size=cfg['img_size'][0], mode='nearest')
#                 pred = model(images)    
                pred = model(images).sigmoid()    
                pred = torch.nn.functional.interpolate(pred, size=256, mode='nearest') 
                preds.append(torch.squeeze(pred, dim=1))
                
                
    else:
        for index, (images, masks) in enumerate(tqdm(valid_loader)):  
            images  = images.to(device, dtype=torch.float)
            masks  = masks.to(device, dtype=torch.float)
            if cfg['img_size'][0] != 256:
                masks = torch.nn.functional.interpolate(masks, size=256, mode='nearest') 

            masks_.append(torch.squeeze(masks, dim=1))

            with torch.inference_mode():
                images = torch.nn.functional.interpolate(images, size=cfg['img_size'][0], mode='nearest')
#                 pred = model(images)   
                pred = model(images).sigmoid()
                pred = torch.nn.functional.interpolate(pred, size=256, mode='nearest') 
                preds.append(torch.squeeze(pred, dim=1))
        
            
            
    model_masks = torch.cat(masks_, dim=0)
    model_preds = torch.cat(preds, dim=0)
        
    model_masks = torch.flatten(model_masks, start_dim=0, end_dim=1)
    model_preds = torch.flatten(model_preds, start_dim=0, end_dim=1)  
    
#     # save
    torch.save(model_preds, f"../../output/final_preds/{cfg['call_sign']}.pt")    
#     if idx == 0:
#         torch.save(model_masks, f'../../output/final_preds/val_masks.pt')

    
    best_threshold = 0.0
    best_dice_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
#         score = dice_coef(model_masks, model_preds.sigmoid(), thr=threshold).cpu().detach().numpy() 
        score = dice_coef(model_masks, model_preds.sigmoid(), thr=threshold).cpu().detach().numpy() 

        if score > best_dice_score:
            best_dice_score = score
            best_threshold = threshold
    
        
    print(best_dice_score, best_threshold)
    final_preds2.append(model_preds)
    
    del model
    torch.cuda.empty_cache()
    gc.collect()

{'model_name': 'Unet', 'backbone': 'tf_mixnet_l', 'img_size': [512, 512], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/Contrail_Detection/output/exp_01/Unet/tf_mixnet_l-512/checkpoint_dice_ctrl_fold0.pth', 'threshold': 0.24, 'model_func': <function load_modelr2 at 0x7f579e7cb9a0>, 'call_sign': 'roh_09_tta_sig', 'tta': True}


  0%|          | 0/52 [00:00<?, ?it/s]

0.62103873 0.57
{'model_name': 'Unet', 'backbone': 'tf_mixnet_l', 'img_size': [512, 512], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/Contrail_Detection/output/exp_01/Unet/tf_mixnet_l-512/checkpoint_dice_ctrl_fold0.pth', 'threshold': 0.24, 'model_func': <function load_modelr2 at 0x7f579e7cb9a0>, 'call_sign': 'roh_09_sig', 'tta': False}


  0%|          | 0/52 [00:00<?, ?it/s]

0.6167561 0.57
{'model_name': 'Unet', 'backbone': 'fbnetv3_g', 'img_size': [256, 256], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/Contrail_Detection/output/exp_01/Unet/fbnetv3_g-256/checkpoint_dice_ctrl_fold0.pth', 'threshold': 0.24, 'model_func': <function load_modelr2 at 0x7f579e7cb9a0>, 'call_sign': 'roh_10_tta_sig', 'tta': True}


  0%|          | 0/52 [00:00<?, ?it/s]

0.60999143 0.53
{'model_name': 'Unet', 'backbone': 'fbnetv3_g', 'img_size': [256, 256], 'num_classes': 1, 'model_pth': '/home/rohits/pv1/Contrail_Detection/output/exp_01/Unet/fbnetv3_g-256/checkpoint_dice_ctrl_fold0.pth', 'threshold': 0.24, 'model_func': <function load_modelr2 at 0x7f579e7cb9a0>, 'call_sign': 'roh_10_sig', 'tta': False}


  0%|          | 0/52 [00:00<?, ?it/s]

0.6041864 0.53


In [21]:
def weighted_ensemble(params, final_preds):    
    for index, val in enumerate(params.keys()):
        if index == 0:
            preds = params[val]*final_preds[0]
        else:
            preds += params[val]*final_preds[index]
    
    param_sum = 0
    for key, val in params.items():
        param_sum += val

    preds = preds/param_sum
    return preds

In [54]:
# # ### Load presaved predictions


def dice_coef2(y_true, y_pred, low_thr=0.5, high_thr=0.5, epsilon=1e-6):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>low_thr).to(torch.float32)
        
#     y_pred = ((y_pred > low_thr) & (y_pred < high_thr)).to(torch.float32)

    
    inter = (y_true*y_pred).sum()
    den = y_true.sum() + y_pred.sum()
    dice = ((2*inter+epsilon)/(den+epsilon)).mean()
    
    return dice



model_masks = torch.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/val_masks.pt")




# call_signs1 = [
#     "nir_tta", "nir_01_tta", "nir_01", "nir_02", "nir_03",
#     "nir_05", "nir_06_tta", "nir_06", "nir_08", "nir_09", "nir_09_tta"
# ]

# nir_02_tta, nir_03_tta, nir_04_tta, roh_03, roh_05_tta, roh_05, roh_06  nir_05_tta, nir_07, nir_04, nir_08_tta


# call_signs1 = [
#     "nir_01", "nir_02", "nir_04",
#     "nir_05_tta", "nir_06", "nir_08"
# ] # later sigmoid

# call_signs1 = [
#     "nir_tta", "nir_01_tta", "nir_01", "nir_02", "nir_03",
#     "nir_05", "nir_06_tta", "nir_06", "nir_08"
# ] # first sigmoid



# call_signs2 = [
#     "roh_01_tta", "roh_01", "roh_03_tta", "roh_04_tta", "roh_04", "roh_05",
#      "roh_06_tta", "roh_07_tta", "roh_07"
# ]

# call_signs2 = [
#     "roh_01_tta", "roh_03_tta", "roh_04_tta", "roh_04", "roh_05",
#     "roh_07_tta", "roh_07"
# ] # later sigmoid

# call_signs2 = [
#     "roh_01_tta", "roh_01", "roh_02_tta", "roh_03_tta", "roh_04_tta",
#      "roh_06_tta", "roh_07_tta"
# ] # first sigmoid


# call_signs1 = [
#     "nir_01", "nir_02", "nir_03", "nir_04",
#     "nir_06_tta", "nir_06", "nir_08_tta", "nir_08", "nir_09_tta"
# ]

# call_signs2 = [
#     "roh_01_tta", "roh_01", "roh_03_tta", "roh_03", "roh_04",
#      "roh_05", "roh_06_tta", "roh_07_tta", "roh_07"
# ]




# call_signs1 = [
#     "nir_01_sig", "nir_02_sig", "nir_03_sig", "nir_04_sig",
#     "nir_06_tta_sig", "nir_06_sig", "nir_08_tta_sig", "nir_08_sig", "nir_09_tta_sig"
# ]

# call_signs1 = [
#     "nir_01_tta_sig", "nir_01_sig", "nir_02_sig",
#     "nir_04_sig", "nir_05_sig",
#     "nir_06_tta_sig", "nir_06_sig", "nir_08_tta_sig", "nir_08_sig",
#     "nir_09_tta_sig", "nir_09_sig"
# ] nir only 688CV

# call_signs1 = [
# #     "nir_01_tta_sig", "nir_01_sig", "nir_02_sig",
# #     "nir_04_sig", "nir_05_sig",
# #     "nir_06_tta_sig", "nir_06_sig", "nir_08_tta_sig", "nir_08_sig",
    
#     "nir_tta_sig",
#     "nir_03_tta_sig",
#     "nir_06_tta_sig",
#     "nir_08_tta_sig",
#     "nir_09_tta_sig"
# ]



# nir_sig, nir_03_sig, nir_07_tta_sig, nir_07_sig
# nir_tta_sig, nir_02_tta_sig, nir_03_tta_sig, nir_04_tta_sig, nir_05_tta_sig

# call_signs2 = [
#     "roh_01_tta_sig", "roh_01_sig", "roh_03_tta_sig", "roh_03_sig", "roh_04_sig",
#      "roh_05_sig", "roh_06_tta_sig", "roh_07_tta_sig", "roh_07_sig"
# ]


# call_signs1 = [
#     "roh_01_tta_sig", "roh_01_sig", "roh_02_tta_sig", "roh_02_sig", "roh_03_tta_sig", "roh_03_sig", "roh_04_tta_sig",
#     "roh_04_sig", "roh_05_tta_sig", "roh_05_sig", "roh_06_tta_sig", "roh_06_sig",  "roh_07_tta_sig", "roh_07_sig",
#     "roh_08_tta_sig", "roh_08_sig", "roh_09_tta_sig", "roh_09_sig" 
# ] # first sigmoid

call_signs1 = [
    "roh_01_tta_sig", "roh_03_tta_sig", "roh_04_tta_sig",
    "roh_05_tta_sig", "roh_05_sig", "roh_06_tta_sig",  "roh_07_tta_sig", "roh_07_sig",
] # first sigmoid


# roh_01_sig, roh_02_sig, roh_03_sig, roh_04_sig, roh_06_sig, roh_08_tta_sig, roh_08_sig


preds1 = []

for idx, sign in tqdm(enumerate(call_signs1), total=len(call_signs1)):
    wt = torch.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/{sign}.pt")    
#     preds1.append(wt.sigmoid())
    preds1.append(wt)
    
    best_threshold = 0.0
    best_dice_score = 0.0
    for threshold in [i / 100 for i in range(101)] :
#         score = dice_coef(model_masks, wt.sigmoid(), thr=threshold).cpu().detach().numpy() 
        score = dice_coef(model_masks, wt, thr=threshold).cpu().detach().numpy() 

        if score > best_dice_score:
            best_dice_score = score
            best_threshold = threshold
    print(best_dice_score, best_threshold)
    
    
    
    
# preds2 = []
    
# for idx, sign in tqdm(enumerate(call_signs2), total=len(call_signs2)):
#     wt = torch.load(f"/home/rohits/pv1/Contrail_Detection/output/final_preds/{sign}.pt")    
# #     preds2.append(wt.sigmoid())
#     preds2.append(wt)
    
    
#     best_threshold = 0.0
#     best_dice_score = 0.0
#     for threshold in [i / 100 for i in range(101)] :
# #         score = dice_coef(model_masks, wt.sigmoid(), thr=threshold).cpu().detach().numpy() 
#         score = dice_coef(model_masks, wt, thr=threshold).cpu().detach().numpy() 
#         if score > best_dice_score:
#             best_dice_score = score
#             best_threshold = threshold
#     print(best_dice_score, best_threshold)

  0%|          | 0/8 [00:00<?, ?it/s]

0.66965246 0.18
0.6132575 0.09
0.68071157 0.11
0.67365956 0.03
0.6682736 0.02
0.68159544 0.21
0.682485 0.19
0.6775493 0.13


In [55]:
# preds1.extend(preds2)   
# final_preds = preds1
# final_preds = torch.stack(final_preds).mean(dim=0)

In [56]:
# final_preds.shape

# best_threshold = 0.0
# best_dice_score = 0.0
# # for threshold in [i / 100 for i in range(101)] :
# for threshold in [0.05, 0.03, 0.032, 0.038, 0.039, 0.04, 0.041, 0.042, 0.045, 0.047, 0.049] :
# #     threshold = 0.5
# #     score = dice_coef(model_masks, final_preds.sigmoid(), thr=threshold).cpu().detach().numpy() 
#     score = dice_coef(model_masks, final_preds.sigmoid(), thr=threshold).cpu().detach().numpy() 
#     if score > best_dice_score:
#         print(threshold, score)
#         best_dice_score = score
#         best_threshold = threshold
# print(best_dice_score, best_threshold)

In [57]:
# 0.6889764
# if len(final_preds) > 0:
#     saved_preds.extend(final_preds)
#     final_preds = saved_preds
# else:
#     final_preds = saved_preds
    
    
    
    
    
# preds1.extend(preds2)   
# final_preds = preds1



In [58]:
final_preds = preds1


In [59]:
# len(final_preds)
# final_preds1.extend(final_preds2)

In [60]:
# final_preds = final_preds1

In [61]:
len(final_preds)

8

In [62]:
def boost_dice(params, TH=None):    
    
    for index, val in enumerate(params.keys()):
        if index == 0:            
            preds = params[val]*final_preds[0]
        else:
            preds += params[val]*final_preds[index]
    
    param_sum = 0
    for key, val in params.items():
        param_sum += val

    preds = preds/param_sum
    
    
    if TH:
        best_threshold = TH
#         preds1 = (preds.sigmoid()).double()
        preds1 = (preds).double()
#         best_score = dice_coef2(model_masks, preds1, low_thr=0.31, high_thr=1.0).cpu().detach().numpy() # get_score(labels, preds) 
        best_score = dice_coef(model_masks, preds1, thr=TH).cpu().detach().numpy() # get_score(labels, preds) 

        
    else:
        best_threshold = 0.0
        best_score = 0.0
        for threshold in [i / 100 for i in range(101)] :              
            if threshold < 0.2 or threshold > 0.65:
                continue
            
#             preds1 = (nn.Sigmoid()(preds)>threshold).double()    
            preds1 = ((preds)>threshold).double()            

            score = dice_coef(model_masks, preds1, thr=threshold).cpu().detach().numpy() # get_score(labels, preds) 
            if score > best_score:
                best_score = score
                best_threshold = threshold
            
#     best_score = score
#     best_threshold = threshold
    return best_score, best_threshold

In [63]:
# final_preds = final_preds1

# len(final_preds)

In [64]:
def objective(trial):
#     TH = 0.29
    TH = None
    
    params = {}    
    for i in range(len(final_preds)):
        params[f"w{i+1}"] = trial.suggest_float(f'w{i+1}', 0, 1) 
        
#     params['w1'] = 0.012132889171043498
# #     params['w1'] = trial.suggest_float(f'w1', 0, 1) 

#     params['w2'] = 0.22910836470987306 
# #     params['w2'] = trial.suggest_float(f'w2', 0, 1) 

#     params['w3'] = 0.8234913030546002 
# #     params['w3'] = trial.suggest_float(f'w3', 0, 1) 

#     params['w4'] = 0.7631962618421504
# #     params['w4'] = trial.suggest_float(f'w4', 0, 1) 

#     params['w5'] = 0.8427707176332744
# #     params['w5'] = trial.suggest_float(f'w5', 0, 1) 

#     params['w6'] = 0.479202185185958
# #     params['w6'] = trial.suggest_float(f'w6', 0, 1) 

#     params['w7'] = 0.416350717272335
# #     params['w7'] = trial.suggest_float(f'w7', 0, 1) 

#     params['w8'] = 0.8815206035791852
# #     params['w8'] = trial.suggest_float(f'w8', 0, 1) 

#     params['w9'] = 0.9217240554494449
# #     params['w9'] = trial.suggest_float(f'w9', 0, 1) 

    
#     params['w10'] = 0.575094040365154
# #     params['w10'] = trial.suggest_float(f'w10', 0, 1) 

#     params['w11'] = 0.26646876984640183
# #     params['w11'] = trial.suggest_float(f'w11', 0, 1) 

#     params['w12'] = 0.11308087290053014 
# #     params['w12'] = trial.suggest_float(f'w12', 0, 1) 

#     params['w13'] = 0.45407230561937456 
# #     params['w13'] = trial.suggest_float(f'w13', 0, 1) 

#     params['w14'] = 0.46687601413910984 
# #     params['w14'] = trial.suggest_float(f'w14', 0, 1) 

#     params['w15'] = 0.5855103418960851 
# #     params['w15'] = trial.suggest_float(f'w15', 0, 1) 


#     params['w16'] = 0.6404939370620236 
# #     params['w16'] = trial.suggest_float(f'w16', 0, 1) 

#     params['w17'] = trial.suggest_float(f'w17', 0, 1) 
#     params['w18'] = trial.suggest_float(f'w18', 0, 1) 



        

#     print(params)
#     exit()

        
    score, best_threshold  = boost_dice(params, TH=TH)
    params['threshold'] = best_threshold
    if not TH:
        print(params)
        
    return score


In [65]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)
best_params = study.best_params


[I 2023-07-04 16:00:19,444] A new study created in memory with name: no-name-c959afe0-24ee-48cd-a5cc-71db01227dc7
[I 2023-07-04 16:00:19,837] Trial 0 finished with value: 0.6867413520812988 and parameters: {'w1': 0.42843914551076867, 'w2': 0.9791606392553401, 'w3': 0.3328923658727868, 'w4': 0.6298746287297864, 'w5': 0.01950059648379221, 'w6': 0.6178454308099809, 'w7': 0.6652680480114249, 'w8': 0.22341379747367585}. Best is trial 0 with value: 0.6867413520812988.


{'w1': 0.42843914551076867, 'w2': 0.9791606392553401, 'w3': 0.3328923658727868, 'w4': 0.6298746287297864, 'w5': 0.01950059648379221, 'w6': 0.6178454308099809, 'w7': 0.6652680480114249, 'w8': 0.22341379747367585, 'threshold': 0.29}


[I 2023-07-04 16:00:20,206] Trial 1 finished with value: 0.6852219700813293 and parameters: {'w1': 0.10639594900644977, 'w2': 0.11731134253367992, 'w3': 0.07394084012691404, 'w4': 0.6900812873863558, 'w5': 0.23695275229704726, 'w6': 0.5598154070183308, 'w7': 0.9726365554757279, 'w8': 0.7745609803555263}. Best is trial 0 with value: 0.6867413520812988.


{'w1': 0.10639594900644977, 'w2': 0.11731134253367992, 'w3': 0.07394084012691404, 'w4': 0.6900812873863558, 'w5': 0.23695275229704726, 'w6': 0.5598154070183308, 'w7': 0.9726365554757279, 'w8': 0.7745609803555263, 'threshold': 0.27}


[I 2023-07-04 16:00:20,576] Trial 2 finished with value: 0.6856850981712341 and parameters: {'w1': 0.07045518345178214, 'w2': 0.05910241516190262, 'w3': 0.09931311263964548, 'w4': 0.23311709747006615, 'w5': 0.42240856437235763, 'w6': 0.6827998142844847, 'w7': 0.24806510494016543, 'w8': 0.6062406215414163}. Best is trial 0 with value: 0.6867413520812988.


...

{'w1': 0.9632954817536054, 'w2': 0.3759835691068409, 'w3': 0.684557178637876, 'w4': 0.3005759327186046, 'w5': 0.7523636915374676, 'w6': 0.1748677080094696, 'w7': 0.3932289278078763, 'w8': 0.8817780863447365, 'threshold': 0.28}


[I 2023-07-04 16:04:05,477] Trial 542 finished with value: 0.6884228587150574 and parameters: {'w1': 0.9207088121423916, 'w2': 0.5333890765316854, 'w3': 0.6031667278687503, 'w4': 0.21167058627908542, 'w5': 0.6913811024373178, 'w6': 0.528573492768651, 'w7': 0.3289090877689639, 'w8': 0.6952212302990795}. Best is trial 349 with value: 0.6886134147644043.


{'w1': 0.9207088121423916, 'w2': 0.5333890765316854, 'w3': 0.6031667278687503, 'w4': 0.21167058627908542, 'w5': 0.6913811024373178, 'w6': 0.528573492768651, 'w7': 0.3289090877689639, 'w8': 0.6952212302990795, 'threshold': 0.28}


[W 2023-07-04 16:04:05,700] Trial 543 failed with parameters: {'w1': 0.9478724475597479, 'w2': 0.46086372579149887, 'w3': 0.7078985535614172, 'w4': 0.3310399786068666, 'w5': 0.02626104453143918, 'w6': 0.7024519175396323, 'w7': 0.41448200390155415, 'w8': 0.26298786093330534} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/rohits/anaconda3/envs/contrail/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_3552747/1553004294.py", line 70, in objective
    score, best_threshold  = boost_dice(params, TH=TH)
  File "/tmp/ipykernel_3552747/3778955076.py", line 34, in boost_dice
    score = dice_coef(model_masks, preds1, thr=threshold).cpu().detach().numpy() # get_score(labels, preds)
KeyboardInterrupt
[W 2023-07-04 16:04:05,702] Trial 543 failed with value None.


KeyboardInterrupt: 

In [77]:
best_params # 0.6887625455856323 | 0.04

# 0.6895816326141357
# {'w1': 0.47086182577836433,
#  'w2': 0.47398487134088524,
#  'w3': 0.5679779129752309,
#  'w4': 0.1252667561239259,
#  'w5': 0.968556649618721,
#  'w6': 0.8607378728752336,
#  'w7': 0.9195324310392224,
#  'w8': 0.7835977002050275,
#  'w9': 0.2781133693982887,
#  'w10': 0.31362551317353027,
#  'w11': 0.705090471708246,
#  'w12': 0.5674062524856331,
#  'w13': 0.9148852628607528,
#  'w14': 0.5186468260988354,
#  'w15': 0.3069714722265997}

{'w1': 0.18401864374824775,
 'w2': 0.4146269962151963,
 'w3': 0.7192886961555446,
 'w4': 0.5539748123446523,
 'w5': 0.7004264938392689}

In [66]:
# dice_coef(model_masks, final_preds, thr=0.05).cpu().detach().numpy()


best_params = {
    'w1': 0.9523918511642355,
    'w2': 0.44974410082629623,
    'w3': 0.6870108787554418,
    'w4': 0.2786414514679672,
    'w5': 0.7394785710916857,
    'w6': 0.49681774450074856,
    'w7': 0.4875680157607395,
    'w8': 0.7471311909621834
}                                                                               
                             
    
                                                                                
                                                                                
                                                                                

In [1]:
# params = {
#  'w1': 0.2250520679228371,
#  'w2': 0.2210497672991009,
#  'w3': 0.1953130079653586,
#  'w4': 0.6793988070403458,
#  'w5': 0.31595425789857745,
#  'w6': 0.6396791846489223,
#  'w7': 0.26004752411514376}


params = {'w1': 0.012132889171043498, 'w2': 0.22910836470987306, 'w3': 0.8234913030546002, 'w4': 0.7631962618421504, 'w5': 0.8427707176332744, 'w6': 0.479202185185958, 'w7': 0.416350717272335, 'w8': 0.8815206035791852, 'w9': 0.9217240554494449, 'w10': 0.575094040365154, 'w11': 0.26646876984640183, 'w12': 0.11308087290053014, 'w13': 0.45407230561937456, 'w14': 0.46687601413910984, 'w15': 0.5855103418960851, 'w16': 0.6404939370620236}





final_preds = weighted_ensemble(params, final_preds)
# torch.save(final_preds, f"../../output/final_preds/0.68866974.pt")

NameError: name 'weighted_ensemble' is not defined

In [39]:
best_threshold = 0.0
best_dice_score = 0.0
# for threshold in [i / 100 for i in range(101)] :
for threshold in [0.025, 0.028, 0.03, 0.038, 0.039, 0.04, 0.041, 0.042, 0.045, 0.047, 0.049] :
#     threshold = 0.5
#     score = dice_coef(model_masks, final_preds.sigmoid(), thr=threshold).cpu().detach().numpy() 
    score = dice_coef(model_masks, final_preds.sigmoid(), thr=threshold).cpu().detach().numpy() 
    if score > best_dice_score:
        best_dice_score = score
        best_threshold = threshold
print(best_dice_score, best_threshold)

0.6889945 0.03


In [80]:
# len(final_preds)

In [29]:
call_signs1.extend(call_signs2)

In [69]:
for idx, val in enumerate(call_signs1):
    print(idx+1, val)

1 roh_01_tta_sig
2 roh_03_tta_sig
3 roh_04_tta_sig
4 roh_05_tta_sig
5 roh_05_sig
6 roh_06_tta_sig
7 roh_07_tta_sig
8 roh_07_sig


In [68]:
best_params # roh_01_sig, roh_02_sig, roh_03_sig, roh_04_sig, roh_06_sig, roh_08_tta_sig, roh_08_sig

{'w1': 0.9523918511642355,
 'w2': 0.44974410082629623,
 'w3': 0.6870108787554418,
 'w4': 0.2786414514679672,
 'w5': 0.7394785710916857,
 'w6': 0.49681774450074856,
 'w7': 0.4875680157607395,
 'w8': 0.7471311909621834}

In [ ]:
# nir_01_tta, nir_02_tta, nir_03_tta, nir_03
# nir_04_tta, roh_03, roh_05_tta
# nir_05, nir_07_tta, roh_02_tta, roh_06
# nir, nir_08_tta
# nir_06_tta, roh_02